In [1]:
import pyodbc
import pandas as pd
import numpy as np
import itertools
import os

In [2]:
work_direc = 'Z:\Capacity & Analytics\Business Informatics\Analytics Team\Surgical Elective Patient Scheduling Model\Theatre_Procedure_Time_Predictive_Modelling\Data_and_model_Nov_2024'
os.chdir(work_direc)
#df = pd.read_excel('InPatient_data_T_O_June_Aug2023.xlsx')

In [3]:
df = pd.read_excel('Theatre_raw_SQL_data_TO_col_2018_2024.xlsx')
df.columns[20:50]

Index(['Session Planned End Date/Time', 'Planned Start Date/Time',
       'Planned Duration', 'Arrived Date/Time', 'Into Theatre Date/Time',
       'Out of Theatre Date/Time', 'Actual Procedure 1 Code 1',
       'Actual Duration', 'H4 Minutes',
       'First Knife to Skin to Last Knife Down Minutes', 'Consultant',
       'Consultant Code', 'Operating Surgeon', 'Specialty', 'Theatre Name',
       'Theatre Suite Name', 'Patient Classification',
       'anaesthetist 1 Surname, Forename', 'anaesthetist 1 Role',
       'anaesthetist 2 Surname, Forename', 'anaesthetist 2 Role',
       'anaesthetist 3 Surname, Forename', 'anaesthetist 3 Role',
       'anaesthetist 4 Surname, Forename', 'anaesthetist 4 Role',
       'anaesthetist Expected?', 'Actual Procedure 1 Code 1.1',
       'Actual Procedure 1 Description 1', 'Actual Procedure 1 Code 2',
       'Actual Procedure 1 Description 2'],
      dtype='object')

In [3]:
from data_processing_nd_encoding_related_functions import age_to_group

def data_prep_first_step_after_SQl_extraction(input_file_name, output_file_name):
    
    def add_count_column(df1, columns_to_count, new_col):
        # Calculate the count of non-missing and non-zero values for each row
        df1[new_col] = df1[columns_to_count].apply(lambda row: (row != 0) & (pd.notna(row))).sum(axis=1)

    def categorise_covid_period(date):
        if date < pd.Timestamp('2020-03-01'):
            return 'pre-covid'
        elif date >= pd.Timestamp('2020-03-01') and date <= pd.Timestamp('2021-03-30'):
            return 'covid'
        else:
            return 'post-covid'
    
    # Check the file extension and read the file accordingly
    if input_file_name.endswith('.csv'):
        df = pd.read_csv(input_file_name)
        print(f"DataFrame loaded from CSV: {input_file_name}")
    elif input_file_name.endswith('.xlsx'):
        df = pd.read_excel(input_file_name, engine='openpyxl')  # Use openpyxl for .xlsx files
        print(f"DataFrame loaded from Excel file: {input_file_name}")
    else:
        raise ValueError("Unsupported file format. Please use a .csv or .xlsx file.")

    
    df = df.replace(['Unknown', '^Not Stated', 'Not Known'], np.nan, regex=True)

    add_count_column(df, ['Actual Procedure 1 Code 1','Actual Procedure 2 Code 1','Actual Procedure 3 Code 1'], 'N_procedures')

    anaesthetic_cols = ['anaesthetist 1 Surname, Forename',
                      'anaesthetist 2 Surname, Forename',
                      'anaesthetist 3 Surname, Forename']

    add_count_column(df, anaesthetic_cols, 'Anaesthetist Count')

    df = df.replace({'ETHNICITY': {'White - Any other White background' : 'Any other White background',
                          'Any Other Ethnic Group': 'Any other ethnic group',
                          'Black African or Black British African' : 'Black or Black British - African',
                         'Any other Black background': 'Black or Black British - Any other Black background',
                          'Black Caribbean or Black British Caribbean':'Black or Black British - Caribbean',
                          'White - British': 'White British',
                          'White - Irish' : 'White Irish',
                          'Mixed - White and Asian':'Mixed White and Asian',
                          'Mixed - White and Black Caribbean' : 'Mixed White and Black Caribbean',
                          'Mixed - Any other background': 'Any other mixed background',
                          'Mixed - White and Black African' : 'Mixed White and Black African',
                          'Pakistani or British Pakistani': 'Asian or Asian British - Pakistani',
                          'Other Ethnic Group - Chinese': 'Asian or Asian British - Chinese',
                          'Chinese': 'Asian or Asian British - Chinese',
                          'Bangladeshi or British Bangladeshi':'Asian or Asian British - Bangladeshi',
                          'Asian - other' : 'Asian or Asian British - Any other Asian background',
                          'Indian or British Indian':'Asian or Asian British - Indian'}})


    white = ['White British', 'White Irish', 'Any other White background']
    other_ethnic_group = ['Any other ethnic group',
                          'Black or Black British - African',
                          'Black or Black British - Any other Black background',
                          'Black or Black British - Caribbean',
                          'Mixed White and Asian',
                          'Mixed White and Black Caribbean',
                          'Any other mixed background',
                          'Mixed White and Black African',
                          'Asian or Asian British - Pakistani',
                          'Asian or Asian British - Chinese',
                          'Asian or Asian British - Chinese',
                          'Asian or Asian British - Bangladeshi',
                          'Asian or Asian British - Any other Asian background',
                          'Any other Asian background',
                          'Asian or Asian British - Indian']
    
    conditions = [df['ETHNICITY'].isin(white), df['ETHNICITY'].isin(other_ethnic_group),
                  df['ETHNICITY'].isna()]
    
    categories = ['Any white background', 'Any other ethnic background', "None"]
    
    df['Ethnicity_grouped'] = np.select(conditions, categories)

    Codes_for_conditions = {
        'IHD': ['I200', 'I201', 'I208', 'I209', 'I210', 'I211', 'I212', 'I213', 'I214', 'I219', 'I220', 'I221', 'I228', 'I229', 
                'I230', 'I231', 'I232', 'I233', 'I234', 'I235', 'I236', 'I238', 'I240', 'I241', 'I248', 'I249', 'I250', 'I251', 'I252', 'I253', 'I254', 'I255', 'I256', 'I258', 'I259'],
        'PAD': ['I7021', 'I7020', 'I7081', 'I7080', 'I7091', 'I7090', 'I72', 'I730', 'I731', 'I738', 
                'I739', 'I742', 'I743', 'I744', 'I745', 'I748', 'I749', 'I77', 'I792', 'I798'],
        'Myocardial infraction': ['I21', 'I22', 'I23', 'I252', 'I258'],
        'Cerebral vascular': ['G450', 'G451', 'G452', 'G454', 'G458', 'G459', 'G46', 'I60',
                              'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I68', 'I69'],
        'Congestive heart failure': ['I500'],
        'Connective tissue disorder': ['M05', 'M060', 'M063', 'M069', 'M32', 'M332', 'M34', 'M353'],
        'Dementia': ['F00', 'F01', 'F02', 'F03', 'F051'],
        'Hypertension': ['I10', 'I11', 'I12', 'I13', 'I15', 'I16'],
        'Diabetes': ['E101', 'E105', 'E106', 'E108', 'E109', 'E111', 'E115', 'E116', 'E118', 'E119', 'E131', 'E135', 'E136', 'E138', 'E139', 'E141', 'E145', 'E146', 'E148', 'E149'],
        'Liver disease': ['K702', 'K703', 'K717', 'K73', 'K74'],
        'Obesity': ['Z68', 'E66'],
        'Peptic ulcer': ['K25', 'K26', 'K27', 'K28'],
        'Periph vasc disease': ['I71', 'I739', 'I790', 'R02', 'Z958', 'Z959']
    }

    secondary_code_cols = df.columns[df.columns.str.startswith('Secondary_Diagnosis')][0:-1:2]

    for key, string_list in Codes_for_conditions.items():
        #print(string_list)
        #default value is 0
        diagnosis_result = np.zeros(len(df))
        for index, row in df.iterrows():
            for diag_code in row[secondary_code_cols]:  # Exclude the last column (NewColumn)
                if pd.isna(diag_code):
                    break
                if any(s in diag_code for s in string_list):
                    diagnosis_result[index]= 1
                    break  # Stop checking if a string is found in any column
        df[key] = diagnosis_result

    df['Age group at admit'] = df['AGE_ON_ADMISSION'].apply(lambda x: age_to_group(x, [5, 15, 30, 50, 60, 70, 80]))

    # Check and proceed line by line
    if 'Arrived Date/Time' in df.columns:
        df['Arrived Date/Time'] = pd.to_datetime(df['Arrived Date/Time'], format="%d/%m/%Y %H:%M")
    
    if 'Out of Theatre Date/Time' in df.columns:
        df['Out of Theatre Date/Time'] = pd.to_datetime(df['Out of Theatre Date/Time'], format="%d/%m/%Y %H:%M")
    
    if 'Arrived Date/Time' in df.columns and 'Out of Theatre Date/Time' in df.columns and 'H4 Minutes' in df.columns:
        df['Out of Theatre Date/Time'] = df.apply(
            lambda row: row['Arrived Date/Time'] + pd.to_timedelta(row['H4 Minutes'], unit='minutes')
            if pd.isnull(row['Out of Theatre Date/Time']) else row['Out of Theatre Date/Time'], 
            axis=1
        )
    
    if 'Discharge Date/Time' in df.columns and 'Admission Date/Time' in df.columns:
        df['Actual LOS'] = (
            pd.to_datetime(df['Discharge Date/Time'], format="%d/%m/%Y %H:%M") - 
            pd.to_datetime(df['Admission Date/Time'], format="%d/%m/%Y %H:%M")
        ).dt.total_seconds() / (24 * 3600)
    
    if 'Discharge Date/Time' in df.columns and 'Out of Theatre Date/Time' in df.columns:
        df['Post-Op LOS'] = (
            pd.to_datetime(df['Discharge Date/Time'], format="%d/%m/%Y %H:%M") - 
            pd.to_datetime(df['Out of Theatre Date/Time'], format="%d/%m/%Y %H:%M")
        ).dt.total_seconds() / (24 * 3600)
    
    if 'Admission Date/Time' in df.columns:
        df['Admission Date/Time'] = pd.to_datetime(df['Admission Date/Time'], format="%d/%m/%Y %H:%M")
        df['Month'] = df['Admission Date/Time'].apply(lambda x: x.strftime('%B'))
        df['Covid Flag'] = df['Admission Date/Time'].apply(categorise_covid_period)
    
    if 'Planned Start Date/Time' in df.columns:
        df['Day of the week'] = pd.to_datetime(df["Planned Start Date/Time"], format="%d/%m/%Y %H:%M").dt.day_name()

    # Check the file extension and save accordingly
    if output_file_name.endswith('.csv'):
        df.to_csv(output_file_name, index=False)
        print(f"DataFrame saved as CSV: {output_file_name}")
    elif output_file_name.endswith('.xlsx'):
        df.to_excel(output_file_name, index=False, engine='openpyxl')  # Use openpyxl for .xlsx files
        print(f"DataFrame saved as Excel file: {output_file_name}")
    else:
        print("Unsupported file format. Please use a .csv or .xlsx file.")

In [5]:
#data_prep_first_step_after_SQl_extraction('Theatre_raw_SQL_data_TO_col_2018_2024.xlsx', 'Theatre_data_prepared_TO_col_2018_2024.csv')
data_prep_first_step_after_SQl_extraction('Theatre_raw_SQL_data_TO_2018_2024.xlsx', 'Theatre_data_prepared_TO_2018_2024.csv')

DataFrame loaded from Excel file: Theatre_raw_SQL_data_TO_2018_2024.xlsx
DataFrame saved as CSV: Theatre_data_prepared_TO_2018_2024.csv


In [6]:
def unique_values(df):
    counts = []
    for col in df.columns:
        temp_name = df[col].value_counts().to_frame()
        temp_name.name = col
        counts.append(temp_name)
        
    return(counts)

In [7]:
#len(df['POSTCODE_LSOA'].unique())

In [6]:
path2 = r'//esneft.nhs.uk/share/Finance/Capacity & Analytics/Business Informatics/Analytics Team/Health Inequalities/Reference Tables'

In [7]:
dod = pd.read_csv(os.path.join(path2, "DoD_LSOA.csv"), encoding_errors="ignore")

In [8]:
dod = dod[['LSOA code (2011)', 'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)',
           'Income Decile (where 1 is most deprived 10% of LSOAs)',
           'Employment Decile (where 1 is most deprived 10% of LSOAs)',
           'Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)',
           'Health Deprivation and Disability Decile (where 1 is most deprived 10% of LSOAs)',
           'Crime Decile (where 1 is most deprived 10% of LSOAs)',
           'Barriers to Housing and Services Decile (where 1 is most deprived 10% of LSOAs)',
          'Living Environment Decile (where 1 is most deprived 10% of LSOAs)' ]]

dod = dod.rename(columns={'Index of Multiple Deprivation (IMD) Decile (where 1 is most deprived 10% of LSOAs)': 'Index of Multiple Deprivation Decile',
           'Income Decile (where 1 is most deprived 10% of LSOAs)':  'Income Decile',
           'Employment Decile (where 1 is most deprived 10% of LSOAs)' : 'Employment Decile',
           'Education, Skills and Training Decile (where 1 is most deprived 10% of LSOAs)' : "Education and Skills Decile" ,
           'Health Deprivation and Disability Decile (where 1 is most deprived 10% of LSOAs)' : 'Health and Disability Decile',
           'Crime Decile (where 1 is most deprived 10% of LSOAs)' :  'Crime Decile',
           'Barriers to Housing and Services Decile (where 1 is most deprived 10% of LSOAs)' : 'Barriers to Housing and Services Decile',
          'Living Environment Decile (where 1 is most deprived 10% of LSOAs)' : 'Living Environment Decile'})

In [9]:
df = df.merge(dod, how='left', left_on='POSTCODE_LSOA', right_on='LSOA code (2011)')

##### REPLACE MISSING VALUES WITH NAN

In [10]:

missing_val = ['Unknown', '^Not Stated', 'Not Known']
df = df.replace(missing_val, np.nan, regex=True)

#missing = df.isna().sum()

In [11]:
df1 = df.copy()

##### COUNT NUMBER OF PROCEDURES

In [12]:

#count number of non empty secondary procedure cells to get total of secondary procedures in each spell
#df['N_Secondary_proc'] = df.iloc[:,78:127:2].apply(lambda x: x.count(), axis=1)

def add_count_column(df1, columns_to_count, new_col):
    # Calculate the count of non-missing and non-zero values for each row
    df1[new_col] = df1[columns_to_count].apply(lambda row: (row != 0) & (pd.notna(row))).sum(axis=1)

procedures_cols = ['Actual Procedure 1 Code 1',
                      'Actual Procedure 2 Code 1',
                      'Actual Procedure 3 Code 1']

add_count_column(df, procedures_cols, 'N_procedures')

In [13]:
#[col for col in df.columns if 'anaes' in col]


In [14]:
anaesthetic_cols = ['anaesthetist 1 Surname, Forename',
                      'anaesthetist 2 Surname, Forename',
                      'anaesthetist 3 Surname, Forename']

add_count_column(df, anaesthetic_cols, 'Anaesthetist Count')

In [15]:
df['Anaesthetist Count'].value_counts()

Anaesthetist Count
2    7606
3    3789
0    2364
1     503
Name: count, dtype: int64

In [18]:
#vals = unique_values(df)

In [19]:
#vals = unique_values(df)

#renaming ethnic groups to ensure consistency (e.g. difference in spelling)
#groups renamed according to 2021 Census - 16 groups in total (with 3 groups not represented in this dataset)
df = df.replace({'ETHNICITY': {'White - Any other White background' : 'Any other White background',
                          'Any Other Ethnic Group': 'Any other ethnic group',
                          'Black African or Black British African' : 'Black or Black British - African',
                         'Any other Black background': 'Black or Black British - Any other Black background',
                          'Black Caribbean or Black British Caribbean':'Black or Black British - Caribbean',
                          'White - British': 'White British',
                          'White - Irish' : 'White Irish',
                          'Mixed - White and Asian':'Mixed White and Asian',
                          'Mixed - White and Black Caribbean' : 'Mixed White and Black Caribbean',
                          'Mixed - Any other background': 'Any other mixed background',
                          'Mixed - White and Black African' : 'Mixed White and Black African',
                          'Pakistani or British Pakistani': 'Asian or Asian British - Pakistani',
                          'Other Ethnic Group - Chinese': 'Asian or Asian British - Chinese',
                          'Chinese': 'Asian or Asian British - Chinese',
                          'Bangladeshi or British Bangladeshi':'Asian or Asian British - Bangladeshi',
                          'Asian - other' : 'Asian or Asian British - Any other Asian background',
                          'Indian or British Indian':'Asian or Asian British - Indian'}})

#df_etn_vals = unique_values(df)

#create another variable with grouped ethnic categories, due to high imbalance in categories 
white = ['White British', 'White Irish', 'Any other White background']
other_ethnic_group = ['Any other ethnic group',
                      'Black or Black British - African',
                      'Black or Black British - Any other Black background',
                      'Black or Black British - Caribbean',
                      'Mixed White and Asian',
                      'Mixed White and Black Caribbean',
                      'Any other mixed background',
                      'Mixed White and Black African',
                      'Asian or Asian British - Pakistani',
                      'Asian or Asian British - Chinese',
                      'Asian or Asian British - Chinese',
                      'Asian or Asian British - Bangladeshi',
                      'Asian or Asian British - Any other Asian background',
                      'Any other Asian background',
                      'Asian or Asian British - Indian']

conditions = [df['ETHNICITY'].isin(white), df['ETHNICITY'].isin(other_ethnic_group),
              df['ETHNICITY'].isna()]

categories = ['Any white background', 'Any other ethnic background', "None"]

df['Ethnicity_grouped'] = np.select(conditions, categories)

#check_ethnicity = unique_values(df)

no_missing = df.replace('None', np.nan)

check_miss = no_missing.isna().sum()



##### STRIP--AND--LOWER--CASE--PROCEDURES

In [21]:

procedure_1 = df.filter(like="Actual Procedure 1 Description")
procedure_2 = df.filter(like="Actual Procedure 2 Description")
procedure_3 = df.filter(like="Actual Procedure 3 Description")

def lower_case(df, cut):
    for col in cut:
        df[col] = df[col].str.lower()
    return(df)

df = lower_case(df, procedure_1)
df = lower_case(df, procedure_2)
df = lower_case(df, procedure_3)

### Adding the Scondary-diagnosis to the columns categorically by exploring into the secondary codes data 

In [22]:
Codes_for_conditions = {
    'IHD': ['I200', 'I201', 'I208', 'I209', 'I210', 'I211', 'I212', 'I213', 'I214', 'I219', 'I220', 'I221', 'I228', 'I229', 
            'I230', 'I231', 'I232', 'I233', 'I234', 'I235', 'I236', 'I238', 'I240', 'I241', 'I248', 'I249', 'I250', 'I251', 'I252', 'I253', 'I254', 'I255', 'I256', 'I258', 'I259'],
    'PAD': ['I7021', 'I7020', 'I7081', 'I7080', 'I7091', 'I7090', 'I72', 'I730', 'I731', 'I738', 
            'I739', 'I742', 'I743', 'I744', 'I745', 'I748', 'I749', 'I77', 'I792', 'I798'],
    'Myocardial infraction': ['I21', 'I22', 'I23', 'I252', 'I258'],
    'Cerebral vascular': ['G450', 'G451', 'G452', 'G454', 'G458', 'G459', 'G46', 'I60',
                          'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I68', 'I69'],
    'Congestive heart failure': ['I500'],
    'Connective tissue disorder': ['M05', 'M060', 'M063', 'M069', 'M32', 'M332', 'M34', 'M353'],
    'Dementia': ['F00', 'F01', 'F02', 'F03', 'F051'],
    'Hypertension': ['I10', 'I11', 'I12', 'I13', 'I15', 'I16'],
    'Diabetes': ['E101', 'E105', 'E106', 'E108', 'E109', 'E111', 'E115', 'E116', 'E118', 'E119', 'E131', 'E135', 'E136', 'E138', 'E139', 'E141', 'E145', 'E146', 'E148', 'E149'],
    'Liver disease': ['K702', 'K703', 'K717', 'K73', 'K74'],
    'Obesity': ['Z68', 'E66'],
    'Peptic ulcer': ['K25', 'K26', 'K27', 'K28'],
    'Periph vasc disease': ['I71', 'I739', 'I790', 'R02', 'Z958', 'Z959']
}


In [23]:
secondary_code_cols = df.columns[df.columns.str.startswith('Secondary_Diagnosis')][0:-1:2]
#secondary_code_cols

In [43]:
#df[Codes_for_conditions.keys()] = 0


In [24]:
# Iterate through the rows and columns to check for the presence of strings
for key, string_list in Codes_for_conditions.items():
    #print(string_list)
    #default value is 0
    diagnosis_result = np.zeros(len(df))
    for index, row in df.iterrows():
        for diag_code in row[secondary_code_cols]:  # Exclude the last column (NewColumn)
            if pd.isna(diag_code):
                break
            if any(s in diag_code for s in string_list):
                diagnosis_result[index]= 1
                break  # Stop checking if a string is found in any column
    df[key] = diagnosis_result


In [25]:
#df['Obesity'].value_counts()
#df.to_csv('Latest_Theatre_data_1.csv')

In [25]:
df['Actual Procedure 1 Code 1'].value_counts()

Actual Procedure 1 Code 1
W409    1475
W903     959
W879     862
W283     676
W379     612
        ... 
T625       1
W596       1
W931       1
W514       1
S575       1
Name: count, Length: 340, dtype: int64

In [23]:
#df['Primary Procedure'].value_counts()

In [45]:
df.head()

,System Key,FACT_IP_SPELLS_ID,System Name,Hosp,Patient ID,AGE_ON_ADMISSION,ETHNICITY,Gender,POSTCODE_LSOA,postcode,...,Obesity,Peptic ulcer,Periph vasc disease,Pulmonary Disease,Diabetes compl,Paraplegia,Renal disease,Metastatic,Severe liver,HIV
0,1000,11950161,Medway,COL,D0305927,66,White British,Female,E01021993,CO7 8NX,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000,11950149,Medway,COL,D0629159,54,White British,Male,E01021715,CO2 9HB,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000,11950148,Medway,COL,D0862257,35,White British,Male,E01021699,CO4 3RH,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000,11950163,Medway,COL,D2016276,56,White British,Female,E01021715,CO2 9RJ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000,11950146,Medway,COL,D6069894,35,White British,Male,E01021677,CO6 1XS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
len(df)

14262

In [47]:
def age_to_group(age, cutoffs = None):
    if cutoffs is None:
        cutoffs = [5, 15, 30, 50, 60, 70, 80]
    for i, cutoff in enumerate(cutoffs):
        if age <= cutoff:
            if i == 0:
                return f"0to{cutoff}"
            elif i == len(cutoffs) - 1:
                return f"{cutoffs[i-1]+1}to{cutoff}"
            else:
                return f"{cutoffs[i-1]+1}to{cutoff}"
    return f"Above{cutoffs[-1]}"

#### Adding extra column with age group

In [48]:
custom_cutoffs = [5, 15, 30, 50, 60, 70, 80]
df['Age group at admit'] = df['AGE_ON_ADMISSION'].apply(lambda x: age_to_group(x, custom_cutoffs))

#### Filling up out of theatre time if missing to calculate post-op LOS 

In [49]:
df['Arrived Date/Time'] = pd.to_datetime(df['Arrived Date/Time'], format="%d/%m/%Y %H:%M")
df['Out of Theatre Date/Time'] = pd.to_datetime(df['Out of Theatre Date/Time'], format="%d/%m/%Y %H:%M")

df['Out of Theatre Date/Time'] = df.apply(lambda row: row['Arrived Date/Time'] + pd.to_timedelta(row['H4 Minutes'], unit='minutes')
                               if pd.isnull(row['Out of Theatre Date/Time']) else row['Out of Theatre Date/Time'], axis=1)


#### Adding Actual LOS and Post-op LOS columns

In [50]:
df['Actual LOS'] = (pd.to_datetime(df['Discharge Date/Time'],format="%d/%m/%Y %H:%M") - pd.to_datetime(df['Admission Date/Time'],format="%d/%m/%Y %H:%M")).dt.total_seconds() / (24 * 3600)

In [51]:
df['Post-Op LOS'] = (pd.to_datetime(df['Discharge Date/Time'],format="%d/%m/%Y %H:%M") - pd.to_datetime(df['Out of Theatre Date/Time'],format="%d/%m/%Y %H:%M")).dt.total_seconds() / (24 * 3600)

#### Adding Day of the week and Month columns

In [52]:
df['Admission Date/Time'] = pd.to_datetime(df['Admission Date/Time'],format="%d/%m/%Y %H:%M")

df['Month'] = df['Admission Date/Time'].apply(lambda x: x.strftime('%B'))
df['Day of the week'] = pd.to_datetime(df["Planned Start Date/Time"],format="%d/%m/%Y %H:%M").dt.day_name()

In [ ]:
#df = pd.read_csv()

#### Adding Covid-Flag data

In [53]:
def categorise_covid_period(date):
    if date < pd.Timestamp('2020-03-01'):
        return 'pre-covid'
    elif date >= pd.Timestamp('2020-03-01') and date <= pd.Timestamp('2021-03-30'):
        return 'covid'
    else:
        return 'post-covid'

In [54]:
df['Covid Flag'] = df['Admission Date/Time'].apply(categorise_covid_period)

In [55]:
theatre_with_laminar_flow = ['Theatre 10', 'Theatre 11', 'Theatre 15']
df['Theatre Airflow System'] = df['Theatre Name'].apply(lambda x: 'Laminar' if x in theatre_with_laminar_flow else 'Non laminar')
df['Theatre category1'] = df['Theatre Suite Name'].apply(lambda x: 'daycase patients' if x == 'Elmstead Theatres' else 'mixed')

#### Exporting to the csv format


In [56]:
df.to_csv('Theatre_data_2018_2024.csv')
#df.to_csv('InPatient_flow_data_inc_diag_encoded_Jun_Aug_2023.csv')

In [46]:
#df['Hosp'].value_counts()

Hosp
COL    11865
IPS     7141
Name: count, dtype: int64

In [47]:
#df['Actual Procedure 1 Code 1.1']

In [53]:
df.columns[-20:]

Index(['Connective tissue disorder', 'Dementia', 'Hypertenstion', 'Diabetes',
       'Liver disease', 'Peptic ulcer', 'Periph vasc disease ',
       'Pulmanory disease', 'Diabetes compl', 'Paraplegia', 'Renal disease',
       'Metastatic', 'Severe liver', 'HIV', 'Age group at admit', 'Actual LOS',
       'Post-Op LOS', 'Month', 'Day of the week', 'Covid Flag'],
      dtype='object')